<a href="https://colab.research.google.com/github/ddzmitry/work_stuff/blob/master/TURTOSETTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

tts = TextToSpeech()

import os
from google.colab import files

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Total 1481 (delta 0), reused 0 (delta 0), pack-reused 1481
Receiving objects: 100% (1481/1481), 53.56 MiB | 26.78 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/tortoise-tts
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.1/819.1 kB 27.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00
  0% (0 of 100715777) |                  | Elapsed Time: 0:00:00 ETA:  --:--:--

Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


In [25]:
# This is the text that will be spoken.
text = "Let's strive to make the world a better place, one code block at a time."

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

# upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "custom"

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
# os.makedirs(custom_voice_folder)
if not os.path.exists(custom_voice_folder):
    os.makedirs(custom_voice_folder)

for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

Saving Sathish1.wav to Sathish1 (2).wav
Saving SathishLoud1.wav to SathishLoud1 (2).wav
Saving SathishLoud2.wav to SathishLoud2 (2).wav


# New Section

In [27]:
# Generate speech
text = "We are doing it right! So excited to show you this deepfake video for leaders!"

# Generate speech
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 16/16 [01:19<00:00,  4.96s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 16/16 [00:23<00:00,  1.47s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [00:59<00:00,  6.72it/s]


In [6]:
!pip install pydub

In [45]:
# import pydub

# Load the audio file
# sound = pydub.AudioSegment.from_file("generated-custom.wav")
import librosa
import soundfile as sf
# Load the audio file
y, sr = librosa.load("generated-custom.wav")

print(sr,y)
# Get the sample rate of the audio file
sample_rate = sr

# Specify the number of semitones to shift the audio file
num_semitones = 0.15
# new_y = librosa.effects.pitch_shift(y, sr, 2)

# Use the librosa.effects.pitch_shift() function to shift the pitch of the audio file
# shifted_audio = librosa.effects.pitch_shift(y, sample_rate, num_semitones)
y_third = librosa.effects.pitch_shift(y, sr=sr, n_steps=num_semitones)
print(y_third)
# Save the shifted audio file
# librosa.output.write_wav("shifted_audio.wav", y_third, sample_rate)

# Load the audio file
# y, sr = librosa.load(y_third)

# Export the audio file as a wav file
sf.write("output.wav", y_third, sr)
IPython.display.Audio(f'output.wav')

22050 [-1.0095462e-03 -8.8514789e-04  1.2245300e-04 ... -5.7001009e-05
 -6.7944653e-05 -3.9636088e-05]
[-9.6015865e-04 -8.7770785e-04  1.4978219e-04 ...  2.2761404e-05
  2.4192643e-05  2.1612828e-05]
